In [1]:
import logging
import os
import sys
import random
import json
from datetime import datetime
module_path = os.path.abspath('/home/alex/data/LightClip/CLIP-KD')
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import torch
from src.open_clip import create_model,AppleMobileCLIP
from src.open_clip.transform import image_transform
from src.open_clip import get_tokenizer

import mobileclip
from mobileclip.models.mci import ParallelAttentionBlock
from PIL import  Image

device = torch.device("cuda")
model = create_model(model_name="timm-swin_tiny_patch4_window7_224",device=torch.device("cuda"))

tokenizer_openclip = get_tokenizer("timm-swin_tiny_patch4_window7_224")
os.environ["CUDA_VISIBLE_DEVICES"] = "3"





model_name:swin_tiny_patch4_window7_224


/home/alex/.conda/envs/clipenv/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
print(model)

CLIP(
  (visual): TimmModel(
    (trunk): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (0): BasicLayer(
          (blocks): Sequential(
            (0): SwinTransformerBlock(
              (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=96, out_features=288, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=96, out_features=96, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop_path): Identity()
              (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (mlp): Mlp(
            

In [2]:
mobile_model, _, preprocess = mobileclip.create_model_and_transforms('mobileclip_s0', pretrained='/home/alex/data/LightClip/ml-mobileclip/checkpoints/mobileclip_s0.pt')
import torchvision.transforms as transforms
preprocess_val = transforms.Compose([
    transforms.Resize(size=256, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(size=(256, 256)),
    transforms.Lambda(lambda image: image.convert('RGB')),  # Assuming _convert_to_rgb is this
    transforms.ToTensor()
])

In [3]:
model = AppleMobileCLIP(**(model.init_params)).to(device)
del model.visual
model.visual = mobile_model.image_encoder.to(device)
del model.transformer
model.transformer = mobile_model.text_encoder.to(device)
del mobile_model

model_name:swin_tiny_patch4_window7_224


In [4]:
image = preprocess_val(Image.open("/home/alex/data/LightClip/ml-mobileclip/docs/fig_accuracy_latency.png").convert('RGB')).unsqueeze(0).half().to(device)
tokenizer = mobileclip.get_tokenizer('mobileclip_s0')

text = tokenizer(["a diagram", "a dog", "a cat"]).to(device)

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)

Label probs: tensor([[1.0000e+00, 1.4368e-06, 5.1833e-07]], device='cuda:0')


In [5]:
print(model.visual.model.network[7][0].token_mixer.qkv)

Linear(in_features=512, out_features=1536, bias=False)


In [6]:
# Assuming the qkv weights are stored as (output_size, input_size * 3)
# Split the weights into q, k, v components
qkv_weight = model.visual.model.network[7][0].token_mixer.qkv.weight

# Determine the size of each component
dim_per_component = qkv_weight.shape[0] // 3
# Extract the v component weights
v_weight = qkv_weight[2*dim_per_component:3*dim_per_component]

print("V component weights:", v_weight)

V component weights: tensor([[ 0.0226, -0.0120,  0.0077,  ...,  0.0017,  0.0294, -0.0240],
        [-0.0075,  0.0069, -0.0092,  ..., -0.0013, -0.0381,  0.0115],
        [ 0.0158,  0.0163, -0.0068,  ..., -0.0035,  0.0162, -0.0020],
        ...,
        [ 0.0015, -0.0030,  0.0181,  ..., -0.0335, -0.0156, -0.0099],
        [-0.0021, -0.0004, -0.0186,  ...,  0.0188,  0.0167,  0.0060],
        [ 0.0025,  0.0045,  0.0761,  ..., -0.0782,  0.0066, -0.0160]],
       device='cuda:0', grad_fn=<SliceBackward0>)


In [9]:
    # Print the main diagonal elements of the v component bias
    v_weight_diagonal = torch.diagonal(v_weight)
    print("V component bias diagonal elements:", v_weight_diagonal)

V component bias diagonal elements: tensor([ 2.2625e-02,  6.9320e-03, -6.7502e-03, -2.2370e-02,  1.0653e-02,
        -7.8609e-03,  1.2283e-02,  2.9718e-03,  9.1769e-03,  8.0343e-04,
        -2.2499e-04,  5.3979e-03, -1.2459e-02,  2.9858e-02,  7.4559e-03,
        -4.7634e-04,  4.7319e-04, -1.0318e-02, -4.5386e-04,  1.7537e-02,
         1.6603e-02,  1.0613e-02,  4.7535e-03, -1.3383e-02, -6.0620e-03,
         3.5502e-02,  1.3125e-02,  3.3291e-02,  1.4827e-02,  5.3697e-03,
         2.0350e-02,  1.5443e-02,  1.8033e-02,  3.3244e-02, -2.0086e-03,
        -2.0147e-02,  1.2509e-02,  2.7069e-02,  1.5394e-02,  8.8825e-03,
        -9.6251e-03,  5.8130e-04,  6.0060e-03,  2.9041e-03, -2.5074e-03,
         1.4419e-02,  9.8336e-03, -1.7645e-02, -1.0508e-02,  2.8049e-03,
         8.0413e-03,  4.3219e-02, -1.3540e-02,  2.6520e-04, -2.6946e-03,
        -1.2744e-02, -6.8052e-03, -1.1126e-02, -1.1560e-03,  1.8180e-02,
        -2.2324e-03, -8.3112e-03,  3.2682e-02,  9.3908e-03, -3.6162e-02,
        -4.8448

In [10]:
# Assuming the qkv weights are stored as (output_size, input_size * 3)
# Split the weights into q, k, v components
qkv_weight = model.visual.model.network[7][1].token_mixer.qkv.weight

# Determine the size of each component
dim_per_component = qkv_weight.shape[0] // 3
# Extract the v component weights
v_weight = qkv_weight[2*dim_per_component:3*dim_per_component]

print("V component weights:", v_weight)

V component weights: tensor([[-0.0282, -0.0370, -0.0082,  ..., -0.0014,  0.0125,  0.0163],
        [ 0.0282,  0.0197,  0.0260,  ...,  0.0127,  0.0022,  0.0008],
        [-0.0160, -0.0287,  0.0116,  ..., -0.0184,  0.0164, -0.0282],
        ...,
        [-0.0005,  0.0175, -0.0311,  ..., -0.0049,  0.0168,  0.0171],
        [ 0.0169, -0.0182,  0.0199,  ..., -0.0044, -0.0176, -0.0370],
        [ 0.0201,  0.0190, -0.0066,  ..., -0.0210, -0.0048, -0.0021]],
       device='cuda:0', grad_fn=<SliceBackward0>)


In [11]:
    # Print the main diagonal elements of the v component bias
    v_weight_diagonal = torch.diagonal(v_weight)
    print("V component bias diagonal elements:", v_weight_diagonal)

V component bias diagonal elements: tensor([-2.8248e-02,  1.9723e-02,  1.1576e-02, -9.1078e-03, -5.5456e-03,
         3.4152e-02,  3.2216e-02,  5.8315e-03, -1.7488e-02, -3.5332e-02,
         2.0467e-02,  5.5233e-03,  2.5466e-03, -3.4352e-03, -4.7485e-03,
        -3.0537e-02,  8.7238e-03,  3.1450e-02, -1.5156e-03, -2.9466e-04,
         2.3092e-02, -5.2767e-04,  4.7456e-02, -8.5221e-03, -2.2598e-02,
         2.0468e-02,  2.5689e-02, -2.5424e-02,  4.8736e-03,  1.8172e-02,
        -4.0648e-02, -2.3025e-02,  1.9518e-02, -6.0889e-03,  4.1003e-03,
        -9.9564e-03, -1.2454e-02, -2.2296e-02, -1.3933e-02,  1.2487e-03,
        -1.0459e-02,  8.5141e-03,  2.3299e-02,  3.9668e-03, -9.4134e-03,
         1.9925e-02, -1.1799e-02,  9.4941e-03, -1.0267e-03, -7.1985e-03,
         2.9697e-02,  1.2786e-02,  4.1704e-03,  1.6534e-02, -2.3965e-02,
        -1.4500e-03,  1.4656e-02, -9.1257e-03, -1.0409e-02, -4.2568e-04,
         1.3144e-02,  4.4290e-03,  3.8770e-03, -2.2936e-02,  1.3292e-03,
        -1.8537